# Termination

在前面几节中，我们介绍了如何定义智能体，并将它们组织为可以解决任务的团队。然而，一个任务的运行可能会无限持续下去，在很多情况下，我们需要知道何时该让它停止——这正是 **终止条件（Termination Condition）** 的作用。

AgentChat 提供了一个基础类 `TerminationCondition`，并基于它实现了多种终止机制。

**终止条件** 本质上是一个可调用对象，它接收自上次调用以来的一组 `BaseAgentEvent` 或 `BaseChatMessage` 消息，并判断是否应该终止对话：

- 如果需要终止，会返回一个 StopMessage；
- 如果不需要终止，则返回 None；
- 一旦某个终止条件被触发，在下一次使用前必须调用 reset() 方法重置它。

注意事项
- 终止条件是有状态的，但在每次运行结束（`run()` 或 `run_stream()`）后会自动重置；
- 支持使用 逻辑与 & 和逻辑或 | 运算符组合多个终止条件；
- 在团队类（如 `RoundRobinGroupChat`、`SelectorGroupChat`、`Swarm`）中，终止条件在每个 agent 回复后调用一次；虽然一个回复中可能包含多个内部消息，但终止条件只针对该 agent 的这组新消息（称为 delta sequence）调用一次。

内置终止条件一览：

| 终止条件                      | 说明                                                     |
| ------------------------- | ------------------------------------------------------ |
| `MaxMessageTermination`   | 当生成的消息数（包括任务和 agent 消息）达到指定数量时终止                       |
| `TextMentionTermination`  | 当消息中包含特定关键词（如 "TERMINATE"）时终止                          |
| `TokenUsageTermination`   | 当使用的提示词或生成词的 token 数超过指定值时终止（需 agent 报告 token 用量）      |
| `TimeoutTermination`      | 当运行时间超过指定秒数时终止                                         |
| `HandoffTermination`      | 当某 agent 发出交接请求（handoff）到指定目标时终止，适用于暂停等待人类接手的场景        |
| `SourceMatchTermination`  | 当特定 agent 回复后终止                                        |
| `ExternalTermination`     | 支持从程序外部手动终止运行，例如 UI 中的“停止”按钮                           |
| `StopMessageTermination`  | 当某 agent 生成 `StopMessage` 时终止                          |
| `TextMessageTermination`  | 当某 agent 生成普通文本消息（`TextMessage`）时终止                    |
| `FunctionCallTermination` | 当某 agent 生成包含特定函数调用结果（`FunctionExecutionResult`）的事件时终止 |
| `FunctionalTermination`   | 当某函数表达式对最新消息序列求值为 `True` 时终止，适用于自定义终止逻辑（灵活创建条件）        |


## Client 配置

In [1]:
import os
from dotenv import load_dotenv
from autogen_ext.models.openai import OpenAIChatCompletionClient

load_dotenv()
siliconflow_api_key = os.getenv("SILICONFLOW_API_KEY") # 读取你的 OPENAI API key

# 初始化 OpenAIChatCompletionClient 客户端，连接到硅基流动平台的 Qwen3-8B 模型
model_client = OpenAIChatCompletionClient(
    model="Qwen/Qwen3-14B",                # 指定要调用的模型名称，硅基流动平台上 Qwen 3-8B 模型
    base_url="https://api.siliconflow.cn/v1",  # 硅基流动平台的 API 访问地址
    api_key=siliconflow_api_key,  # 你的 API 密钥
    model_info={                        
        "family": "qwen",              
        "context_length": 8192,        
        "max_output_tokens": 2048,     
        "tool_choice_supported": True, 
        "tool_choice_required": False,  
        "structured_output": True,     
        "vision": False,                
        "function_calling": True,      
        "json_output": True           
    },
)

为了演示终止条件的特点，我们将创建一个由两个智能体组成的团队：

- 主智能体（primary agent）：负责生成文本内容；

- 评论智能体（critic agent）：负责对生成的文本进行审阅并提供反馈。

通过这种设置，我们可以观察在不同终止条件（例如关键词触发、评论通过、外部中止等）下，团队如何决定何时停止运行，从而实现可控的多智能体协作流程。

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# 假设 model_client 已在前面初始化
model_client = model_client

# 创建主智能体
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# 创建评论智能体
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback for every message. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# 示例1：使用最大消息数终止条件
max_msg_termination = MaxMessageTermination(max_messages=3)
round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=max_msg_termination)

# 运行团队任务，输出消息到控制台
await Console(round_robin_team.run_stream(task="Write a unique, Haiku about the weather in Paris"))

# 示例2：使用关键词终止条件
# text_termination = TextMentionTermination("APPROVE")
# round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)
# await Console(round_robin_team.run_stream(task="Write a unique, Haiku about the weather in Paris"))

---------- TextMessage (user) ----------
Write a unique, Haiku about the weather in Paris
---------- ThoughtEvent (primary) ----------

Okay, the user wants a unique haiku about the weather in Paris. Let me start by recalling what a haiku is. It's a traditional Japanese poem with three lines, syllable structure 5-7-5. So first line 5 syllables, second 7, third 5.

Now, Paris weather... I need to think about the typical weather there. Paris is known for its variable weather. Maybe mention the Seine River, Eiffel Tower, or Montmartre. But the user wants something unique, so I shouldn't just use clichés. Let me brainstorm some imagery.

Autumn in Paris has leaves falling, maybe a bit of rain. Spring could be blooming flowers, but that's common. What about a specific scene? Like a café with steam from coffee, or fog around the Eiffel Tower. Fog is a good image, it's not too usual. Maybe combine that with the Eiffel Tower's silhouette. 

First line: "Fog clings to Eiffel's" – that's 5 sylla

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a unique, Haiku about the weather in Paris', type='TextMessage'), ThoughtEvent(source='primary', models_usage=None, metadata={}, content='\nOkay, the user wants a unique haiku about the weather in Paris. Let me start by recalling what a haiku is. It\'s a traditional Japanese poem with three lines, syllable structure 5-7-5. So first line 5 syllables, second 7, third 5.\n\nNow, Paris weather... I need to think about the typical weather there. Paris is known for its variable weather. Maybe mention the Seine River, Eiffel Tower, or Montmartre. But the user wants something unique, so I shouldn\'t just use clichés. Let me brainstorm some imagery.\n\nAutumn in Paris has leaves falling, maybe a bit of rain. Spring could be blooming flowers, but that\'s common. What about a specific scene? Like a café with steam from coffee, or fog around the Eiffel Tower. Fog is a good image, it\'s not too usual. May

让我们来看一下终止条件是如何在每次调用 `run()` 或 `run_stream()` 后**自动重置**的。
这使得团队可以在终止后**从中断的地方继续对话**，而不会受到上一次终止状态的影响。

具体来说：

* 每次团队运行结束（不论是自然结束还是被终止），绑定的终止条件对象会**自动调用 `reset()`**；
* 当你再次调用 `run()` 或 `run_stream()` 时，终止条件会重新开始评估，不会“记住”上次的触发状态；
* 团队内部的对话上下文（如历史消息）仍然被保留，因此团队可以从上次中断的地方继续运行。

这意味着你可以设计一个**逐轮推进式的交互流程**：
运行一轮 → 用户反馈 → 继续运行 → 再反馈 → …

这种行为特别适用于：

* 多轮对话控制；
* 分阶段任务执行；
* 需要插入人类反馈的场景（Human-in-the-Loop）。


## 最大轮数停止

当特定数量的消息被生成之后，自动终结对话。  
***本文中涉及到消息数量的终止条件指的是TextMessage,ThoughEvent并不算做消息***

In [ ]:
# 示例1：使用最大消息数终止条件
max_msg_termination = MaxMessageTermination(max_messages=3)
round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=max_msg_termination)

# 如果你在脚本中运行，请用 asyncio.run(...) 包裹主函数
# 这里直接运行团队任务，并将消息流输出到控制台
await Console(round_robin_team.run_stream(task="Write a unique, Haiku about the weather in Paris"))

---------- TextMessage (user) ----------
Write a unique, Haiku about the weather in Paris
---------- ThoughtEvent (primary) ----------

Okay, the user wants a unique haiku about the weather in Paris. Let me start by recalling what a haiku is. It's a traditional Japanese poem with three lines, syllable structure 5-7-5. So I need to make sure each line fits that.

First, I should think about Paris's weather. It's known for being mild, with lots of rain, especially in spring and autumn. Maybe mention the Seine River, since that's a iconic feature. Also, the cityscape with buildings and maybe the Eiffel Tower. The contrast between the gray skies and the beauty of the city could be a good angle.

Let me brainstorm some images. Rain on the Seine, fog around the Eiffel Tower, the smell of bread from a boulangerie. Those are all Parisian elements. Now, how to fit them into 5-7-5 syllables.

First line: "Rain whispers on Seine" – that's 5 syllables. Good. Second line needs 7. Maybe something ab

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a unique, Haiku about the weather in Paris', type='TextMessage'), ThoughtEvent(source='primary', models_usage=None, metadata={}, content='\nOkay, the user wants a unique haiku about the weather in Paris. Let me start by recalling what a haiku is. It\'s a traditional Japanese poem with three lines, syllable structure 5-7-5. So I need to make sure each line fits that.\n\nFirst, I should think about Paris\'s weather. It\'s known for being mild, with lots of rain, especially in spring and autumn. Maybe mention the Seine River, since that\'s a iconic feature. Also, the cityscape with buildings and maybe the Eiffel Tower. The contrast between the gray skies and the beauty of the city could be a good angle.\n\nLet me brainstorm some images. Rain on the Seine, fog around the Eiffel Tower, the smell of bread from a boulangerie. Those are all Parisian elements. Now, how to fit them into 5-7-5 syllables

对话在达到最大消息数限制后停止了。由于主智能体（primary agent）尚未回应评论智能体的反馈，我们可以选择继续对话，让任务顺利完成。

In [3]:
# 如果你在脚本中运行，请用 asyncio.run(...) 包裹主函数
# 这里直接运行团队任务，并将消息流输出到控制
await Console(round_robin_team.run_stream())

---------- ThoughtEvent (primary) ----------

Okay, let's see. The user wants a Haiku about the weather in Paris. The original one I wrote was:

Seine whispers through rain,
Paris breathes in twilight's gentle haze—
Cafés steam, then fade.

But they pointed out some issues. First, the syllable structure. The second line is 6 syllables instead of 7. Let me count again. "Paris breathes in twilight's gentle haze—" That's Paris (2), breathes (1), in (1), twilight's (2), gentle (2), haze (1). Wait, maybe I miscounted. Let me break it down:

- "Paris breathes in" = 3 syllables (Paris, breathes, in)
- "twilight's gentle haze—" = 5 syllables (twi-light's, gen-tle, haze)

So total is 3 + 5 = 8. Hmm, maybe I got that wrong before. The user said 6, but perhaps they counted differently. Let me check again. Maybe "twilight's" is pronounced as two syllables (twi-light's) but when hyphenated, they might split it as one? No, that's not right. "Twilight's" is still two syllables. Wait, maybe the user c

TaskResult(messages=[ThoughtEvent(source='primary', models_usage=None, metadata={}, content='\nOkay, let\'s see. The user wants a Haiku about the weather in Paris. The original one I wrote was:\n\nSeine whispers through rain,\nParis breathes in twilight\'s gentle haze—\nCafés steam, then fade.\n\nBut they pointed out some issues. First, the syllable structure. The second line is 6 syllables instead of 7. Let me count again. "Paris breathes in twilight\'s gentle haze—" That\'s Paris (2), breathes (1), in (1), twilight\'s (2), gentle (2), haze (1). Wait, maybe I miscounted. Let me break it down:\n\n- "Paris breathes in" = 3 syllables (Paris, breathes, in)\n- "twilight\'s gentle haze—" = 5 syllables (twi-light\'s, gen-tle, haze)\n\nSo total is 3 + 5 = 8. Hmm, maybe I got that wrong before. The user said 6, but perhaps they counted differently. Let me check again. Maybe "twilight\'s" is pronounced as two syllables (twi-light\'s) but when hyphenated, they might split it as one? No, that\'s 

## Combining Termination Conditions

我们来演示如何使用 与运算符 (&) 和 或运算符 (|) 将多个终止条件组合在一起，以构建更复杂的终止逻辑。



示例目标
创建一个团队，满足以下任一条件时终止运行：

- 消息总数达到 10 条，或

- 评论智能体（critic agent）对某条消息给予 “APPROVE” 的肯定评价。

In [12]:
max_msg_termination = MaxMessageTermination(max_messages=10)  # 当消息总数达到10条时终止
text_termination = TextMentionTermination("APPROVE")          # 当消息中出现 "APPROVE" 时终止
combined_termination = max_msg_termination | text_termination # 组合终止条件，满足任一条件即终止

round_robin_team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=combined_termination
)

# 如果你在脚本中运行，请用 asyncio.run(...) 包裹主函数
# 运行团队任务，并将消息流输出到控制台
await Console(round_robin_team.run_stream(task="Write a unique, Haiku about the weather in Paris"))

---------- TextMessage (user) ----------
Write a unique, Haiku about the weather in Paris
---------- ThoughtEvent (primary) ----------

Okay, the user asked for another unique Haiku about the weather in Paris. Let me start by recalling the previous one. They had "Rain whispers on Seine, Fog cloaks the Eiffel in quiet haze— Café warmth dissolves it." And they approved of that after some feedback.

So, I need to come up with something different. Let me think about the key elements that define Parisian weather. Paris is known for its rainy days, foggy mornings, mild temperatures, and maybe the contrast between the city's architecture and the weather.

First line: Focus on a sensory detail. Maybe involve the Seine again, but not exactly the same as before. "Rain whispers" was good, but perhaps another metaphor. Like "mist dances on Pont des Arts" to include a bridge and movement.

Second line: The Eiffel Tower is iconic. "Fog cloaks" was used, maybe something with the locket or its silhoue

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a unique, Haiku about the weather in Paris', type='TextMessage'), ThoughtEvent(source='primary', models_usage=None, metadata={}, content='\nOkay, the user asked for another unique Haiku about the weather in Paris. Let me start by recalling the previous one. They had "Rain whispers on Seine, Fog cloaks the Eiffel in quiet haze— Café warmth dissolves it." And they approved of that after some feedback.\n\nSo, I need to come up with something different. Let me think about the key elements that define Parisian weather. Paris is known for its rainy days, foggy mornings, mild temperatures, and maybe the contrast between the city\'s architecture and the weather.\n\nFirst line: Focus on a sensory detail. Maybe involve the Seine again, but not exactly the same as before. "Rain whispers" was good, but perhaps another metaphor. Like "mist dances on Pont des Arts" to include a bridge and movement.\n\nSeco

可以看到对话在评论智能体（critic agent）批准消息后停止了，尽管如果消息数量达到 10 条，它也会终止运行。

如果我们希望只有在两个条件都满足时才终止（例如：“消息数量 ≥ 10”且“收到 APPROVE”），就需要使用逻辑与 & 运算符。

在这个组合中，只有当：

- 消息总数达到或超过 10 条，并且

- 评论者明确回复了 “APPROVE”，

才会终止

In [14]:
max_msg_termination = MaxMessageTermination(max_messages=10)  # 当消息总数达到10条时终止
text_termination = TextMentionTermination("APPROVE")          # 当消息中出现 "APPROVE" 时终止
combined_termination = max_msg_termination | text_termination # 组合终止条件，满足任一条件即终止

round_robin_team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=combined_termination
)

# 如果你在脚本中运行，请用 asyncio.run(...) 包裹主函数
# 运行团队任务，并将消息流输出到控制台
await Console(round_robin_team.run_stream(task="Write a unique, Haiku about the weather in Paris"))

---------- TextMessage (user) ----------
Write a unique, Haiku about the weather in Paris


---------- ThoughtEvent (primary) ----------

Okay, the user asked for another unique haiku about the weather in Paris. Let me see. They previously had a haiku where I suggested "Café warmth dissolves it." Now they want a fresh take.

First, I need to recall what a haiku is: three lines with a 5-7-5 syllable structure. Paris weather can be rainy, foggy, maybe a bit chilly, but also the contrast with human elements like cafes or people. 

The user might appreciate imagery that's iconic yet not overused. They liked the previous focus on Seine, Eiffel Tower, and cafes. Maybe avoid "Eiffel" again to keep it unique. Think of other Parisian elements like bridges, parks, streets, or maybe something like the Louvre or Montmartre.

I should incorporate seasonal aspects. Paris is known for its spring and fall weather, which is mild. Maybe the transition between seasons? Or the way light changes during different times of the day?

Sensory details are important. The previous one had "whispers" and

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a unique, Haiku about the weather in Paris', type='TextMessage'), ThoughtEvent(source='primary', models_usage=None, metadata={}, content='\nOkay, the user asked for another unique haiku about the weather in Paris. Let me see. They previously had a haiku where I suggested "Café warmth dissolves it." Now they want a fresh take.\n\nFirst, I need to recall what a haiku is: three lines with a 5-7-5 syllable structure. Paris weather can be rainy, foggy, maybe a bit chilly, but also the contrast with human elements like cafes or people. \n\nThe user might appreciate imagery that\'s iconic yet not overused. They liked the previous focus on Seine, Eiffel Tower, and cafes. Maybe avoid "Eiffel" again to keep it unique. Think of other Parisian elements like bridges, parks, streets, or maybe something like the Louvre or Montmartre.\n\nI should incorporate seasonal aspects. Paris is known for its spring an

## Custom Termination Condition

内置的终止条件已经覆盖了大多数使用场景，但有时你可能需要实现一个不属于现有终止条件的自定义逻辑。这时，你可以通过继承 `TerminationCondition` 类，来创建属于你自己的终止条件。

🛠 示例：自定义终止条件（监听特定函数调用）
下面这个例子演示了如何创建一个终止条件，当某个特定函数被调用时，自动停止对话。

In [3]:
from typing import Sequence

from autogen_agentchat.base import TerminatedException, TerminationCondition
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage, StopMessage, ToolCallExecutionEvent
from autogen_core import Component
from pydantic import BaseModel
from typing_extensions import Self

class FunctionCallTerminationConfig(BaseModel):
    """用于序列化和反序列化终止条件组件的配置类。"""
    function_name: str

class FunctionCallTermination(TerminationCondition, Component[FunctionCallTerminationConfig]):
    """
    当检测到指定名称的 FunctionExecutionResult（即函数调用结果）时，终止对话。
    用于实现“某个函数被调用后自动停止团队运行”的自定义终止条件。
    """

    component_config_schema = FunctionCallTerminationConfig
    component_provider_override = "autogen_agentchat.conditions.FunctionCallTermination"

    def __init__(self, function_name: str) -> None:
        self._terminated = False
        self._function_name = function_name

    @property
    def terminated(self) -> bool:
        """指示终止条件是否已被触发。"""
        return self._terminated

    async def __call__(self, messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> StopMessage | None:
        """
        检查消息序列中是否有 ToolCallExecutionEvent，且其 name 等于指定函数名。
        如果检测到，则返回 StopMessage 并设置终止状态；否则返回 None。
        """
        if self._terminated:
            raise TerminatedException("Termination condition has already been reached")
        for message in messages:
            if isinstance(message, ToolCallExecutionEvent):
                for execution in message.content:
                    if execution.name == self._function_name:
                        self._terminated = True
                        return StopMessage(
                            content=f"Function '{self._function_name}' was executed.",
                            source="FunctionCallTermination",
                        )
        return None

    async def reset(self) -> None:
        """重置终止条件状态，使其可再次使用。"""
        self._terminated = False

    def _to_config(self) -> FunctionCallTerminationConfig:
        """用于序列化配置。"""
        return FunctionCallTerminationConfig(
            function_name=self._function_name,
        )

    @classmethod
    def _from_config(cls, config: FunctionCallTerminationConfig) -> Self:
        """用于反序列化配置。"""
        return cls(
            function_name=config.function_name,
        )

我们来使用这个新的自定义终止条件，使团队在评论智能体（critic agent）调用名为 approve 的函数时自动终止对话。



第一步：定义 approve 函数（将被 agent 调用）

In [4]:
def approve() -> None:
    """Approve the message when all feedbacks have been addressed."""
    pass


接下来，我们创建两个智能体（agents），其中评论智能体（critic agent）绑定了 approve 工具，用于对内容进行审核并在合适时调用 approve() 表示批准，从而触发终止条件。

In [5]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# 假设 model_client 已在前面初始化
model_client = model_client

# 创建主智能体
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# 创建评论智能体，并注册 approve 工具
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    tools=[approve],  # 注册 approve 函数为工具
    system_message="Provide constructive feedback. Use the approve tool to approve when all feedbacks are addressed.",
)

接下来，我们将完成以下步骤：

- 创建自定义终止条件（当评论智能体调用 approve() 函数时终止）
- 创建团队并应用该终止条件
- 启动任务，让团队合作生成一首诗

In [6]:
function_call_termination = FunctionCallTermination(function_name="approve")
round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=function_call_termination)

# 如果你在脚本中运行，请用 asyncio.run(...) 包裹主函数
await Console(round_robin_team.run_stream(task="Write a unique, Haiku about the weather in Paris"))


---------- TextMessage (user) ----------
Write a unique, Haiku about the weather in Paris
---------- ThoughtEvent (primary) ----------

Okay, the user wants a unique haiku about the weather in Paris. Let me start by recalling what a haiku is. It's a traditional Japanese poem with three lines, syllable structure 5-7-5.

First, I need to think about the typical weather in Paris. Paris is known for its temperate climate, with mild winters and warm summers. But to make it unique, maybe I should focus on a specific aspect or a particular season that's not too common. For example, the drizzle in spring or the October mist. Wait, the user didn't specify a season, so maybe a general weather feature that's characteristic of Paris.

I should also include elements that evoke imagery related to Paris—like the Seine, Eiffel Tower, cafés, or maybe the cobbled streets. How to combine the weather with these elements in a concise way.

Let me brainstorm some words related to Paris weather. Drizzle, fog

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a unique, Haiku about the weather in Paris', type='TextMessage'), ThoughtEvent(source='primary', models_usage=None, metadata={}, content='\nOkay, the user wants a unique haiku about the weather in Paris. Let me start by recalling what a haiku is. It\'s a traditional Japanese poem with three lines, syllable structure 5-7-5.\n\nFirst, I need to think about the typical weather in Paris. Paris is known for its temperate climate, with mild winters and warm summers. But to make it unique, maybe I should focus on a specific aspect or a particular season that\'s not too common. For example, the drizzle in spring or the October mist. Wait, the user didn\'t specify a season, so maybe a general weather feature that\'s characteristic of Paris.\n\nI should also include elements that evoke imagery related to Paris—like the Seine, Eiffel Tower, cafés, or maybe the cobbled streets. How to combine the weather

你可以看到，当评论智能体（critic agent）通过调用 approve 函数批准了生成的内容后，对话就立即终止了。

这是因为我们设置的自定义终止条件 FunctionNameTermination("approve") 捕捉到了这次函数调用事件，并触发了 StopMessage，从而干净地结束了团队运行。

✅ 这一流程展示了以下关键特性：

- 函数级终止控制：通过监听特定的工具/函数调用，可以实现精确控制对话何时结束；
- 终止条件可扩展：即使内置条件无法满足需求，也能通过继承 TerminationCondition 灵活实现；
- 与工具调用无缝协作：AutoGen 的工具系统与终止机制高度兼容，适合构建自定义 agent 控制逻辑。